In [1]:
import tensorflow as tf
import numpy as np

import Poetry
import cn_utils

%load_ext autoreload
%autoreload 2

In [2]:
poetries = cn_utils.readFile('./poetry')

word_to_idx, idx_to_word = cn_utils.get_idx_and_words(poetries)

all files:
['\xe6\xb8\xb8\xe5\xad\x90\xe5\x90\x9f.txt', '\xe8\x8d\x89.txt', '\xe5\x85\xb3\xe5\xb1\xb1\xe6\x9c\x88.txt', '\xe7\x99\xbb\xe9\xb9\xb3\xe9\x9b\x80\xe6\xa5\xbc.txt', '\xe7\x9b\xb8\xe6\x80\x9d.txt', '\xe7\xbb\x88\xe5\x8d\x97\xe6\x9c\x9b\xe4\xbd\x99\xe9\x9b\xaa.txt', '\xe7\x99\xbb\xe4\xb9\x90\xe6\xb8\xb8\xe5\x8e\x9f.txt', '\xe9\xb9\xbf\xe6\x9f\xb4.txt', '\xe9\x9d\x99\xe5\xa4\x9c\xe6\x80\x9d.txt', '\xe6\x9d\x82\xe8\xaf\x97.txt', '\xe5\xbc\xb9\xe7\x90\xb4.txt', '\xe5\x85\xab\xe9\x98\xb5\xe5\x9b\xbe.txt', '\xe6\xb1\x9f\xe9\x9b\xaa.txt', '\xe6\x98\xa5\xe6\x99\x93.txt']
reading file:  ./poetry/游子吟.txt
游子吟
stt慈母手中线，游子身上衣。临行密密缝，意恐迟迟归。谁言寸草心，报得三春晖。end
----- reading finished -----


reading file:  ./poetry/草.txt
草
stt离离原上草，一岁一枯荣。野火烧不尽。春风吹又生。远芳侵古道，晴翠接荒城。又送王孙去，萋萋满别情。end
----- reading finished -----


reading file:  ./poetry/关山月.txt
关山月
stt明月出天山，苍茫云海间。长风几万里，吹度玉门关。汉下白登道，胡窥青海湾。由来征战地，不见有人还。戍客望边色，思归多苦颜。高楼当此夜，叹息未应闲。end
----- reading finished -----


reading file:  ./poetry/登鹳雀楼.txt
登鹳雀楼
stt白日依山尽

In [3]:
encoded_poetries = []
for pt in poetries:
    encoded_poetries.append(cn_utils.encodePoetry(pt, word_to_idx))

In [4]:
import poetry_utils
data = {}
data = poetry_utils.generate_data(encoded_poetries, data)

In [5]:
type(encoded_poetries[0].title[0])

int

In [6]:
dataset = Poetry.Poetries(encoded_poetries)

poetry length:  41
poetry length:  51
poetry length:  77
poetry length:  30
poetry length:  28
poetry length:  31
poetry length:  30
poetry length:  28
poetry length:  29
poetry length:  28
poetry length:  28
poetry length:  29
poetry length:  28
poetry length:  28


In [7]:
dataset.data_set

array([[  3,   4,   5, ...,   7,   8,   9],
       [  4,   5,   1, ...,   8,   9,  10],
       [  5,   1,   6, ...,   9,  10,  11],
       ..., 
       [ 45, 257, 258, ..., 259, 205, 108],
       [257, 258,  11, ..., 205, 108, 260],
       [258,  11, 211, ..., 108, 260,  15]], dtype=int32)

In [8]:
dataset[0:1][0].shape

(1, 8)

# 建立tensorflow图

In [9]:
tf.reset_default_graph()
batch_size = 8
vocab_size = len(idx_to_word)
hidden_size = 200
num_steps = dataset.sample_len
lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_size)
cell = tf.contrib.rnn.MultiRNNCell([lstm_cell] * 2)

input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
targets = tf.placeholder(tf.int32, [batch_size, num_steps])

embedding = tf.get_variable(name='embedding', shape=[len(idx_to_word), hidden_size])

inputs = tf.nn.embedding_lookup(embedding, input_data)
outputs = []

initial_state = cell.zero_state(batch_size, tf.float32)

state = initial_state

with tf.variable_scope('RNN'):
    for time_step in range(dataset.sample_len):
        if time_step > 0: tf.get_variable_scope().reuse_variables()
        cell_output, state = cell(inputs[:,time_step,:], state)
        outputs.append(cell_output)
        
output = tf.reshape(tf.concat(outputs, 1), [-1, hidden_size])

weight = tf.get_variable('weight', [hidden_size, vocab_size])
bias = tf.get_variable('bias', [vocab_size])
logits = tf.matmul(output, weight) + bias

loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(targets, [-1])],
                                                          [tf.ones([batch_size * num_steps], dtype = tf.float32)])
cost = tf.reduce_sum(loss) / batch_size

final_state = state

trainable_variables = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, trainable_variables), clip_norm=5)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0)
train_op = optimizer.apply_gradients(zip(grads, trainable_variables))

# 训练

In [50]:
sess = tf.InteractiveSession()

_state = sess.run(initial_state)
sess.run(tf.global_variables_initializer())
T = 5000
predict = None

for i in xrange(T):

    idx = np.random.randint(low = 0, high = dataset.data_set.shape[0], size = (batch_size))
    _cost, _state, _ = sess.run([cost,final_state, train_op], 
                              feed_dict = {input_data: dataset.data_set[idx,:].reshape(batch_size,-1),
                                           initial_state : _state,
                                           targets: dataset.target[idx,:].reshape(batch_size,-1)
                                           })
    if i % 100 == 0:
        print 'The %dth iteration...\n' % i, 'the cost is: ', _cost


test_data = np.array(cn_utils.encodeChinese('白日依山尽blkblkblk', word_to_idx), dtype = np.int32).reshape(1,-1)
test_data = np.concatenate([test_data] * 8, axis = 0)

predict = sess.run(logits, feed_dict = {input_data:test_data})



The 0th iteration...
the cost is:  44.5229
The 100th iteration...
the cost is:  34.5241
The 200th iteration...
the cost is:  20.5002
The 300th iteration...
the cost is:  10.8449
The 400th iteration...
the cost is:  6.7156
The 500th iteration...
the cost is:  6.52918
The 600th iteration...
the cost is:  5.00978
The 700th iteration...
the cost is:  4.06611
The 800th iteration...
the cost is:  3.00028
The 900th iteration...
the cost is:  4.76069
The 1000th iteration...
the cost is:  2.83523
The 1100th iteration...
the cost is:  2.70327
The 1200th iteration...
the cost is:  5.34258
The 1300th iteration...
the cost is:  1.96612
The 1400th iteration...
the cost is:  2.65002
The 1500th iteration...
the cost is:  1.0482
The 1600th iteration...
the cost is:  2.41097
The 1700th iteration...
the cost is:  3.62368
The 1800th iteration...
the cost is:  2.72209
The 1900th iteration...
the cost is:  2.75193
The 2000th iteration...
the cost is:  2.71793
The 2100th iteration...
the cost is:  2.08698
Th

# 预测测试，可以看出有那么点样子了。
目前还是只能预测一句话，而且格式得中规中矩，每句都得5个字，而且逗号句号都得标出来。还没想好怎么生成完整的古诗

In [53]:
test_data = np.array(cn_utils.encodeChinese('八阵图功盖三分国', word_to_idx), dtype = np.int32).reshape(1,-1)
test_data = np.concatenate([test_data] * 8, axis = 0)

predict = sess.run(logits, feed_dict = {input_data:test_data})
ans = np.argmax(predict, axis = 1)

ans = ans[:8]

print cn_utils.decodeChinese(ans, idx_to_word)

阵图。盖三分国，


In [10]:
from IPython.display import clear_output, Image, display, HTML

# Helper functions for TF Graph visualization

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
show_graph(tf.get_default_graph())